In [ ]:
!pip install datasets evaluate transformers
!pip install contractions
!pip install word2number
!pip install spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 5.3 MB/s 
     |████████████████████████████████| 69 kB 5.4 MB/s 
     |████████████████████████████████| 4.7 MB 50.2 MB/s 
     |████████████████████████████████| 120 kB 70.0 MB/s 
     |████████████████████████████████| 115 kB 75.8 MB/s 
     |████████████████████████████████| 212 kB 70.0 MB/s 
     |████████████████████████████████| 127 kB 72.6 MB/s 
     |████████████████████████████████| 6.6 MB 3.2 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 106 kB 4.4 MB/s 
     |████████████████████████████████| 287 kB 34.0 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.d

In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertForSequenceClassification, TFBertTokenizer
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np

from tabulate import tabulate
from tqdm import trange
import random
from torch import nn
import re
import nltk
from nltk.corpus import stopwords
import spacy
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer 
from word2number import w2n
import re
from tqdm import tqdm
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
# spacy.load('en_core_web_lg')
spacy.load('en_core_web_sm')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
ds2 = pd.read_csv('tech_edited_4.csv', sep='|')
df = ds2.copy()

FileNotFoundError: ignored

In [ ]:
# добавляем название статьи в начало текста
df['text'] = df['title'] + ' ' + df['text'] 

In [ ]:
df = df.rename(columns={'transport': 'label'})

In [ ]:
# no <=> 0 <=> no transport
# yes <=> 1 <=> transport
df = df.replace({'no': 0, 'yes': 1})

In [ ]:
len(df)

In [ ]:
from random import randint
rand_ind = randint(0, 100)

In [ ]:
text = df.text.values
labels  = df.label.values

In [ ]:
x_true = df[df['label'] == 1].text.values
y_true = df[df['label'] == 1].label.values

In [ ]:
def get_true_false_data(train_size=600):
    all_text = [] 
    all_label = []
    false_labels = 0
    true_labels = 0
    all_test_text = []
    all_label_text = []
    # выбираем данные, классифицированные как transport и как no transport
    x_true = df[df['label'] == 1].text.values
    y_true = df[df['label'] == 1].label.values
    x_false = df[df['label'] == 0].text.values[100:]
    y_false = df[df['label'] == 0].label.values[100:]
    # try to get train_size + 50 samples data with 
    # пробуем 
    for x_now, y_now in zip(x_true, y_true):
        if true_labels >= train_size + 50:
            break
        if true_labels <= train_size and y_now == 1:
            all_text.append(x_now)
            all_label.append(y_now)
            true_labels += 1
        elif true_labels > train_size + 10 and y_now == 1:
            all_test_text.append(x_now)
            all_label_text.append(y_now)
            true_labels += 1
    for x_now, y_now in zip(x_false, y_false):
        if false_labels >= train_size + 50:
            break
        if false_labels <= train_size and y_now == 0:
            all_text.append(x_now)
            all_label.append(y_now)
            false_labels += 1
        elif false_labels <= train_size + 10 and y_now == 0:
            all_test_text.append(x_now)
            all_label_text.append(y_now)
            false_labels += 1
    x = all_text + all_test_text
    y = all_label + all_label_text
    return (x, y)

In [ ]:
text, labels = get_true_false_data()

In [ ]:
len(labels)

In [ ]:
labels = np.array(list(map(int, labels)))

In [ ]:
tokenizer = BertTokenizer.from_pretrained(
    'bert-base-uncased',
    do_lower_case = True
    )
token_id = []
# attention_masks - приводим все семплы к одной длине
attention_masks = []

#создаем функцию токенизации с помощью encode_plus
def preprocessing(input_text, tokenizer):
  return tokenizer.encode_plus(
      input_text,
      add_special_tokens  = True,
      max_length = 512, 
      pad_to_max_length = True,
      return_attention_mask = True,
      return_tensors = 'pt'
  )

#записываем в словарь необходимые значние:attention_masks и token_id 
for sample in text:
  encoding_dict = preprocessing(sample, tokenizer)
  token_id.append(encoding_dict['input_ids']) 
  attention_masks.append(encoding_dict['attention_mask'])


token_id = torch.cat(token_id, dim = 0)
attention_masks = torch.cat(attention_masks, dim = 0)
labels = torch.tensor(labels)


In [ ]:
test_size = 0.2
batch_size = 16
range_labels = np.arange(len(labels))

train_indexes, validation_indexes = train_test_split(
    range_labels,
    test_size = test_size,
    # shuffle = True,
    stratify = labels
)

In [ ]:

# создаем dataset с тренировочной и с валидационной выборкой
train_set = TensorDataset(
    token_id[train_indexes], 
    attention_masks[train_indexes], 
    labels[train_indexes]
)

val_set = TensorDataset(
    token_id[validation_indexes], 
    attention_masks[validation_indexes], 
    labels[validation_indexes]
)


train_dataloader = DataLoader(
    train_set, 
    sampler = RandomSampler(train_set), 
    batch_size = batch_size
)

validation_dataloader = DataLoader(
            val_set,
            sampler = SequentialSampler(val_set),
            batch_size = batch_size
        )

In [ ]:
def b_tp(preds, labels):
  '''Returns True Positives (TP): количество правильных предсказаний класса 1'''
  return sum([preds == labels and preds == 1 for preds, labels in zip(preds, labels)])

def b_fp(preds, labels):
  '''Returns False Positives (FP): количество неправильных предсказаний класса 1'''
  return sum([preds != labels and preds == 1 for preds, labels in zip(preds, labels)])

def b_tn(preds, labels):
  '''Returns True Negatives (TN): количество правильных предсказаний класса 0'''
  return sum([preds == labels and preds == 0 for preds, labels in zip(preds, labels)])

def b_fn(preds, labels):
  '''Returns False Negatives (FN): количество неправильных предсказаний класса 0'''
  return sum([preds != labels and preds == 0 for preds, labels in zip(preds, labels)])

def b_metrics(preds, labels):
  '''
    - accuracy    = (TP + TN) / N
    - precision   = TP / (TP + FP)
    - recall      = TP / (TP + FN)
    - specificity = TN / (TN + FP)
  '''
  # выбираем индекс класса, для которого вероятность больше
  preds = np.argmax(preds, axis = 1).flatten()
  labels = labels.flatten()
  # считаем метрики
  true_positive = b_tp(preds, labels)
  true_negative = b_tn(preds, labels)
  false_positive = b_fp(preds, labels)
  false_negative = b_fn(preds, labels)
  b_accuracy = ((true_positive + true_negative) / len(labels)) if len(labels) > 0 else 'nan'
  b_precision = true_positive / (true_positive + false_positive) if (true_positive + false_positive) > 0 else 'nan'
  b_recall = true_positive / (true_positive + false_negative) if (true_positive + false_negative) > 0 else 'nan'
  b_specificity = true_negative / (true_negative + false_positive) if (true_negative + false_positive) > 0 else 'nan'
  return b_accuracy, b_precision, b_recall, b_specificity



In [ ]:
model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels = 2,
    output_attentions = False,
    output_hidden_states = False,
)


In [ ]:
optimizer = torch.optim.AdamW(
    model.parameters(), 
    # optimizer_grouped_parameters,
    lr = 3e-5,
    eps = 1e-08,
    weight_decay = 1e-2
)
model.cuda()

In [ ]:
from transformers.utils.dummy_pt_objects import DEBERTA_V2_PRETRAINED_MODEL_ARCHIVE_LIST
# переводим все на gpu
device = torch.device('cuda')
epochs = 4
loss_values = []
mean_accuracy = []
for _ in trange(epochs):
  model.train()
  train_loss = 0
  nb_tr_steps = 0
  for _, batch in enumerate(train_dataloader):
        # переводим весь батч на gpu
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        # обнуляем произвдные (так как optimizer их сохраняет)
        optimizer.zero_grad()
        # Forward pass
        train_output = model(
            b_input_ids, 
            token_type_ids = None, 
            attention_mask = b_input_mask, 
            labels = b_labels
            )
        # считаем градиент
        train_output.loss.backward()
        optimizer.step()
        # Обновляем лосс и считаем шаги
        train_loss += train_output.loss.item()
        nb_tr_steps += 1
  model.eval()
  # Tracking variables 
  val_accuracy = []
  val_precision = []
  val_recall = []
  val_specificity = []
  # пробегаемся по validation_dataloader и считаем метрики
  for batch in validation_dataloader:
      batch = tuple(t.to(device) for t in batch)
      b_input_ids, b_input_mask, b_labels = batch
      # отключаем градиент для вывода eval_output
      with torch.no_grad():
        eval_output = model(
            b_input_ids, 
            token_type_ids = None, 
            attention_mask = b_input_mask
            )
      logits = eval_output.logits.detach().cpu().numpy()
      label_ids = b_labels.to('cpu').numpy()
      print(logits, label_ids)
      # Calculate validation metrics
      b_accuracy, b_precision, b_recall, b_specificity = b_metrics(logits, label_ids)
      val_accuracy.append(b_accuracy)
      # Update precision only when (tp + fp) !=0; ignore nan
      if b_precision != 'nan': val_precision.append(b_precision)
      # Update recall only when (tp + fn) !=0; ignore nan
      if b_recall != 'nan': val_recall.append(b_recall)
      # Update specificity only when (tn + fp) !=0; ignore nan
      if b_specificity != 'nan': val_specificity.append(b_specificity)
  tr_loss = train_loss / nb_tr_steps
  val_accuracy_now = sum(val_accuracy) / len(val_accuracy)
  val_precision_now = sum(val_precision) / len(val_precision) if len(val_precision) > 0 else '\t - Validation Precision: NaN'
  val_recall_now = sum(val_recall) / len(val_recall) if len(val_recall) > 0 else '\t - Validation Recall: NaN'
  val_specificity_now = sum(val_specificity) / len(val_specificity) if len(val_specificity) > 0 else '\t - Validation Specificity: NaN'
  loss_values.append(train_loss / nb_tr_steps)
  mean_accuracy.append(val_accuracy_now)
  print('\n\t - Train loss: {:.4f}'.format(tr_loss))
  print('\t - Validation Accuracy: {:.4f}'.format(val_accuracy_now))
  print('\t - Validation Precision: {}'.format(val_precision))
  print('\t - Validation Recall: {}'.format(val_recall))
  print('\t - Validation Specificity: {:.4f}\n'.format(val_specificity_now))



In [ ]:
import matplotlib.pyplot as plt

plt.plot(loss_values)
plt.show()

In [ ]:
import matplotlib.pyplot as plt

plt.plot(mean_accuracy)
plt.show()

In [ ]:
def get_probability(text):
  new_sentence = text

  # We need Token IDs and Attention Mask for inference on the new sentence
  test_ids = []
  test_attention_mask = []
  # Apply the tokenizer
  encoding = preprocessing(new_sentence, tokenizer)
  # Extract IDs and Attention Mask
  test_ids.append(encoding['input_ids'])
  test_attention_mask.append(encoding['attention_mask'])
  test_ids = torch.cat(test_ids, dim = 0)
  test_attention_mask = torch.cat(test_attention_mask, dim = 0)

  # Forward pass, calculate logit predictions
  with torch.no_grad():
    output = model(test_ids.to(device), token_type_ids = None, attention_mask = test_attention_mask.to(device))
  prediction = 'transport' if np.argmax(output.logits.cpu().numpy()).flatten().item() == 1 else 'no transport'
  # print("---")
  # print(output)
  # print(torch.nn.functional.softmax(output.logits.cpu())) # ham / spam
  # print("---")
  # print('Input Sentence: ', new_sentence)
  # print('Predicted Class: ', prediction)
  probs = torch.nn.functional.softmax(output.logits.cpu()) 
  return prediction

# My tests

In [ ]:
txt = '''John Carmack, the game developer who co-founded id Software and served as Oculus’s CTO, is working on a new venture — and has already attracted capital from some big names.

Carmack said Friday his new artificial general intelligence startup, called Keen Technologies (perhaps a reference to id’s “Commander Keen“), has raised $20 million in a financing round from former GitHub CEO Nat Friedman and Cue founder Daniel Gross. Stripe co-founder and CEO Patrick Collison, Shopify co-founder and CEO Tobi Lütke, and storied venture fund Sequoia are among those investing in the round, he said.
“This is explicitly a focusing effort for me. I could write a $20M check myself, but knowing that other people’s money is on the line engenders a greater sense of discipline and determination. I had talked about that as a possibility for a while, and I am glad Nat pushed me on it,” he said in a tweet.
AGI, short for “artificial general intelligence,” is a category of AI systems that can theoretically perform any task that a human can. That’s as opposed to AI systems today, which are designed for narrow, specific applications like generating art, driving cars, and playing video games. Some optimists believe AGI could be achieved within the next century, thanks to emerging algorithmic techniques and increasingly powerful computer hardware. Others, including the chief scientist at Carmack’s current employer, Yann LeCun, have cast doubt on the notion that AGI will ever be a reality.
For his part, Carmack seems firmly in the optimist camp. Speaking to MIT research scientist Lex Fridman on a recent podcast, he said that he believes AI will someday reach the point where it behaves akin to a human being or living creature, capable of becoming a “universal remote worker.”
“I do not believe in fast takeoffs … [but] I think that we will go from a point where we start seeing [AI-powered] things that credibly look like animals,” Carmack said. “I think animal intelligence is closer to human intelligence than a lot of people like to think and … There’s just that smooth spectrum of how the brain developed and cortexes and scaling of different things going on there.”
Carmack goes on to note that his interest in AGI was in part spurred by Sam Altman, who he says tried to recruit him to join OpenAI. (Altman is one of the original co-founders of OpenAI, the high-profile AI research lab based in San Francisco.) Altman, who’s previously said he expects AI to one day surpass human intelligence, has been vocal about his concerns about AGI — specifically how it might damage society if built or used incorrectly.
Carmack will still continue to consult with Meta on VR matters, but his work with the social juggernaut, he said, will consume only 20% of his time. Carmack moved to a “consulting CTO” position at Oculus in November 2019 to devote more time to researching AI.
Carmack isn’t the only storied technologist to found an ambitious AI startup recently. For roughly a year, LinkedIn co-founder Reid Hoffman and founding DeepMind member Mustafa Suleyman have been building Inflection AI, a startup that seeks to translate human interactions into a language computers can understand. Former VP of research at OpenAI Dario Amodei is among the co-founders of Anthropic, which for most of 2021 and into this year has studied the behaviors of AI text-generating systems like OpenAI’s GPT-3.'''

In [ ]:
txt2 = '''SoftBank-backed Socar, South Korea’s largest car-sharing startup, tumbles on market debut Socar, South Korea’s largest car-sharing startup, tumbled in its Seoul stock market debut Monday even after pricing shares below the bottom end of a marketed range.

Shares of Socar rose 1.25% from its IPO price of 28,000 won ($21.10) in the initial minutes of the debut, before tumbling to 26,300 won and giving the firm a market cap of $642 million.

Last week, Socar cut its targeted IPO offering to 102 billion won ($78.1 million), giving the car-sharing company a valuation of 966.5 billion won ($731 million) before the start of trading.


Socar’s debut comes amid a sluggish period in the IPO market in South Korea that has prompted a series of Korean companies to delay their listing plans.

Analysts attributed the muted performance of Socar’s debut day to an expensive valuation and slowdown in the IPO market that is reeling from the global economic downturn.

Jaeuk Park, CEO of Socar, told TechCrunch earlier that the company was pushing ahead with its listing plans because it was confident in its performance and expected to generate both operating profits and net profits by the end of this year.

Rather than waiting for the stock market to recover for a higher valuation for the next two to three years, he said, the Korean mobility startup prioritized investment in organic/inorganic growth with the IPO proceeds.

“First off, Socar’s growth is faster than expected in the face of re-opening [after the COVID-19 pandemic],” Park said. “The stock market is expected to be tough for the time being, but the mobility industry will grow rapidly that we can’t miss this critical time; we will focus on accelerating mergers and acquisitions and investments in new businesses and technology.”


The company plans to beef up its services and geographic expansion efforts via acquisitions to become a mobility super-app with the goal of posting 1 trillion won ($748 million) of revenue by 2025, up from 289 billion won last year, Park continued. It aims to reach 30% or more annual revenue growth by 2025, he added.

Socar, the first and only unicorn mobility startup in South Korea, aims to become the first listed profitable unicorn company, Park noted.

Socar, backed by SoftBank, and Korean strategic investor SK Inc., entered the unicorn club with roughly 183.2 billion won ($150 million) funding at a 1.3 trillion won (approximately $1 billion) in March from Lotte Rental, the rental car unit of South Korean Lotte Group. The startup raised a total of 379.7 billion won ($ 284.2million) since its inception in 2011 before its IPO.

The company’s major shareholders, including SoftBank, SK Inc., Lotte Rental and Altos Ventures, will hold their stake since they agreed to a lock-up period of up to six months.


The 11-year-old firm, which started the car-sharing service with 100 rental cars in Jeju, now manages a fleet of more than 19,000 vehicles across the country, offering services, including car-sharing, car-hailing, electric bicycle rental, parking, vehicle management and vehicle maintenance. It will launch its transportation super-app later this year, providing all-in-one mobility services. Additionally, Socar is building an ecosystem for future mobility, including an autonomous driving platform, charging stations service for electric cars and micromobility.

Park said in an interview that Socar wants to enter the Southeast Asia market with its new business, a fleet management system (FMS) B2B SaaS service that it plans to begin selling later in the fourth quarter of this year. Socar, based on its 19,000 vehicles, has built the FMS technology that uses data, such as vehicle location and surroundings to support effective monitoring and control systems, providing accurate information to drivers and management servers.

“FMS is different from car-sharing, which has been Socar’s flagship business for the past ten years, and if stabilized, it is a B2B SaaS that guarantees a high-profit ratio,” Park said.

Socar claims that the company has captured about 80% of the market share in South Korea, with more than 11.4 million users and 1.4 million monthly active users this year.


The Korean car-hailing company has set up Socar Malaysia, a 79% stake owned by SK Inc., and launched services in Malaysia in 2018 and Indonesia in 2020.

Socar was founded in 2011 by Lee Jae-woong, who co-founded South Korea’s largest internet portal operator Daum Communications; Daum merged into Kakao in 2014. Jaeuk Park, a serial entrepreneur, who founded VCNC, an operator messaging app for couples called Between, in 2011, sold VCNC to Socar in 2018. After the sale of VCNC, Park joined as Socar’s chief strategy officer to lead Socar’s ride-hailing business Tada and took on the chief executive officer role (CEO) in 2020 after Lee stepped down.

Korean game maker Krafton acquired VCNC’s messaging app unit in May 2021, while Viva Republica, a Korean financial super app Toss operator, bought a 60% stake in Tada, the ride-hailing business of VCNC, for an undisclosed amount in October last year.

Meanwhile, South Korea’s TMap Mobility, whose investors include Uber Technologies and SK Inc.’s investment firm SK Square, said Monday it has raised $149.2 million (200 billion won) from strategic investor KB Bank. Another Korean car-hailing platform Kakao Mobility, which also had planned an IPO between 2022 and 2023, said last week it has terminated its sale talk with Korean private equity firm MBK Partners.'''

In [ ]:
txt3 = '''Bangalore-based Exponent Energy might have come up with a way to deliver 15-minute rapid charging for electric vehicles. The startup, which just raised a $13 million Series A, relies on a combination of its proprietary battery pack and charging infrastructure to achieve such a feat.

Exponent Energy’s business model is geared toward OEMs building commercial EVs for fleet purposes. Ideally, the company works with the OEM to integrate its battery pack, or e^pack, that can then be charged quickly via Exponent’s network of chargers, or e^pumps. Earlier this month, Exponent announced its first partnership with Altigreen, an Indian electric cargo vehicle manufacturer, launching the Exponent-enabled Altigreen neEV HD, a three-wheeler that both companies say can be fully charged, from 0% to 100%, in 15 minutes.

The rub is that the battery pack only charges that quickly when it’s being charged on Exponent’s charging infrastructure — if the e^pack is being charged at a standard charging station, it’ll take about 60 minutes, according to the company. Likewise, the e^pumps don’t deliver the same rapid charge to all EVs, so the two must be scaled side-by-side. This is how Exponent hopes to monetize its energy offerings. It will earn revenue from both the sale of the battery pack to OEMs and the charging on a recurring basis, according to Arun Vinayak, Exponent’s co-founder and CEO.


For comparison, Exponent’s business model is a somewhat similar model to Gogoro, the Taiwanese company that works with OEMs to integrate its swappable batteries into their electric two-wheelers while simultaneously building out swapping stations around the country.

Aside from the monetization logic of including the battery pack and charging infrastructure as a package deal, Vinayak says it just makes sense to do so from a technology perspective.

“15-min rapid charging is a two-sided problem,” said Vinayak. “It’s not just the battery but also the charger. The e^pump delivers 600A of current to the e^pack (15x industry standard) while managing individual cell characteristics, including thermals to ensure safety, long battery life and performance consistency, even at 50 degrees Celsius. Since our technology is present on both sides, we’re able to manage the flow of energy far more efficiently, safely and rapidly.”

Building out such a network will require funds, which is where Exponent’s Series A comes in. The round, which was led by Lightspeed with participation from YourNest VC, 3one4 Capital and AdvantEdge VC, will be used to scale up the e^pump network to 100 location points in each city Exponent expands into, starting with Bengaluru and eventually making its way to New Delhi, according to Vinayak. The company also aims to deploy 2,000 Exponent-enabled vehicles as part of its partnership with Altigreen.'''

In [ ]:
txt4 = '''Motional, the Aptiv-Hyundai joint venture that’s working to commercialize autonomous driving technology, has launched its new all-electric IONIQ 5-based robotaxi for driverless ride-hail operations on the Lyft network in Las Vegas. The news follows years of collaboration between the two companies in Las Vegas.

The launch of the IONIQs, which marks the turnover of Motional’s fleet of BMW-based AVs, is right on schedule. Motional and Lyft said last November that they would aim to start transporting passengers using the Hyundai vehicles in the City of Sin by the second half of 2022, with a full-scale commercial launch set for 2023.

Similar to Motional’s autonomous ride-hailing service that the company launched with Via in February, riders will not be charged for autonomous Lyft rides — the companies are mutually focused on rider feedback, said a company spokesperson. The spokesperson also noted that Motional has a permit to conduct fully driverless testing anywhere in Nevada, and that Motional and Lyft will secure the appropriate permits to start conducting commercial rides in fully driverless vehicles ahead of the launch in 2023.


The Nevada Department of Motor Vehicles has not responded to TechCrunch’s requests for clarification on what permits those might be, but a spokesperson from the department previously told TechCrunch the state is working on drafting new legislation around AVs. Nevada’s current legislation doesn’t make a distinction between testing or operating AVs with a human safety operator onboard or without, nor does it include specific language around commercializing robotaxi or autonomous delivery.

Motional did not share how many IONIQ 5s will be deployed in Las Vegas, but the company did say it will continue to expand its fleet over the course of 2022 as it gears up for its commercial launch next year.

Any Lyft rider in Las Vegas can request a Motional AV, according to the spokesperson. There will be two vehicle operators present in the front seats — presumably one behind the steering wheel in case a human needs to take over, and one in the passenger seat to record data and observations. The service route is focused on high-traffic and popular destinations along the Vegas strip and will only be available during daytime hours, which is fair when one considers that not many AV systems are trained to spot and avoid drunk hooligans vomiting on the street.

Customers who order a robotaxi ride can use the Lyft app to unlock the doors of the autonomous IONIQs. While inside the vehicle, riders can interact with an in-ride display that allows them to start the ride or contact customer support. Many of the features within the vehicles come from years of testing and rider feedback from over 100,000 rides over the last four years, according to Motional.

Motional has said for a time that it plans to expand to other major U.S. cities after commercially launching in Las Vegas, but the company hasn’t said which other cities it is targeting. Like 47 other companies, Motional has a permit to test its driverless technology in California, a state that has become a key battleground for companies like Cruise and Waymo. Indeed, in May, Motional began a delivery pilot with Uber Eats in Santa Monica, and in July the company expanded its autonomous testing in San Diego. However, Motional might look further afield for less competition. Late last year, Motional began increasing public road testing in its hometown of Boston, and in order to really stand out amongst the competition, putting an AV service on the icy streets of New England might be just the right challenge.

'''

In [ ]:
txt5 = '''Helbiz started out as a shared micromobility company but has since expanded to include ghost kitchens, media streaming and, most recently, a taxi service. The company reported its second-quarter earnings Monday after the bell. The startup was the first scooter operator to go public via the SPAC route, and many in the industry wish it wasn’t so after consistently meh earnings reports.

Since Helbiz’s public debut in August 2021, its earnings reports have shown a company that burns through dwindling cash reserves, doesn’t pull in enough revenue to make up for its high costs of operations and keeps pivoting away from core operations into new, and sometimes strange, business units.

While Helbiz’s revenue has increased slightly quarter over quarter and year over year, Monday’s report tells a similar story.


Before we dig into the financials, a little context. In late June, Helbiz signed a letter of intent to buy Wheels, another shared micromobility operator, by the end of the year. In the midst of this, there were multiple times when Helbiz employees in U.S. and Serbian offices had to wait for delayed payments. Sources told TechCrunch that aside from late paychecks, Helbiz is suffering from chronically late scooter shipments and a general lack of company structure.

Despite lackluster earnings, Helbiz’s stock is trading higher than its public market rival Bird, which also announced earnings today. Today, at $1.43 after hours, Helbiz is up 12.6%. That is largely attributable to Helbiz CEO Salvatore Palella’s acquisition of 252,636 shares of the company at an average price of $3 — a transaction that is valued at $757,908. Also, that number is still a far cry from the $10.92 at which Helbiz opened.

Helbiz’s Q2 2022 Financials
Helbiz closed out the second quarter with $4.4 million in revenue, which is up 46% from the same period last year and 33% from last quarter. Mobility, or shared micromobility rides, made up more than half of the second quarter’s total revenue at $2.7 million, up from $1.6 million in Q1.

Helbiz reported around 1.2 million rides in Q2, which is nearly double its Q1 rides, but only a slight increase YoY. Unlike Bird, Helbiz doesn’t appear to report the number of vehicles it has on the ground, nor its rides per vehicle per day.

The remaining $1.7 million in revenue came from “the incremental contribution from Media and Kitchen,” said Helbiz chief financial officer Giulio Profumo in a statement.


During Q3 2021, Helbiz launched Helbiz Live, a sports streaming platform that is currently showing Italy’s Series B soccer, NCAA football and basketball, and MLB games. Helbiz expects to generate $6 million during the first Series B season, some of which must have already been realized in Q2 2022.

Around the same time that Helbiz launched Live, it also introduced Helbiz Kitchen, a ghost kitchen delivery service. The company was coy about how much revenue the new service has brought in, but Kitchen apparently delivered something. Helbiz said in the first half of the year, revenue nearly doubled sequentially. Of course, doubled from zero isn’t exactly a massive achievement.

“Importantly, growth was solid in our core mobility business and we are improving margins as we bring down mobility cost of revenue,” said Profumo. “Even with our cost-control focus, we are investing effectively and efficiently in talent, advertising, marketing, and R&D to sustain our pace of expansion.”

Helbiz’s operating expenses did decrease slightly QoQ, but at $20.8 million, they nearly doubled YoY. Loss from operations was down at $16.4 million from $18 million in Q1, but Helbiz’s net loss of $19.7 million is about flat QoQ.


The company finished the quarter with $2.5 million in cash, which is up from $1 million last quarter, but way down from $21 million during the same period last year. Helbiz had to raise $10 million this quarter via a new issue of convertible notes. In July and August, Helbiz also raised another $5 million to fund its “multiple growth opportunities,” according to Profumo.

The first half of the year saw Helbiz use about $4.7 million in cash to fund its micromobility operations. The company paid $3.5 million to vehicle manufacturers as deposits for e-bikes, e-scooters and e-mopeds, vehicles that Helbiz expects to be delivered throughout the year. And while Helbiz’s acquisition of Wheels will be mainly stock, Helbiz put down a $1 million deposit to enter into the letter of intent, and invested $100,000 in operating licenses, which it has categorized as intangible assets.

“Looking forward, we will deploy more vehicles, pursue more micro-mobility licenses, and drive expansion in Asia Pacific,” said the CFO. Helbiz recently launched shared e-scooter operations in Australia and expanded its existing fleets in the U.S. and Italy.

The company provided no guidance for the third quarter or the full year.'''

In [ ]:
txt6 = '''The Station is a weekly newsletter dedicated to all things transportation. Sign up here — just click The Station — to receive it every weekend in your inbox.

Welcome back to The Station, your central hub for all past, present and future means of moving people and packages from Point A to Point B. 

Concerns and controversy continue to swirl around Tesla’s so-called Full Self-Driving software beta.

It seems that in the Tesla-Twittersphere we’ve reached peak absurdity (although check back with me next week) with a well-known booster attempting to show the abilities of the software by asking for people to volunteer their children run in front of the vehicle. This Twitter account can be difficult to pin down as some tweets are purposely audacious, while others are serious and reflect actual beliefs.


I don’t have much to say about this since anyone who follows the most basic tenets of society and uses logic understands it is wrong and ridiculous. My one thought: Perhaps, it’s time for us all to put our grown-up pants on and stop normalizing this kind of behavior.

Against that backdrop of unsafe practices, I wanted to note an important departure over at the Department of Transportation. Steven Cliff, the head of the National Highway Traffic Safety Administration, plans to leave the agency in September after three months in the top job. NHTSA chief counsel Ann Carlson will take over for now. Cliff is headed over to the California Air Resources Board.

Safety agencies have already pushed the White House to fill the spot ASAP. Why? It has sat vacant since 2017.

You can email me at kirsten.korosec@techcrunch.com to share thoughts, criticisms, opinions or tips. You also can send a direct message to @kirstenkorosec.

Micromobbin’
the station scooter1a

The North American Bikeshare and Scooter Association released its 2021 shared micromobility state of the industry report — and it has some interesting findings.


One positive takeaway: Shared micromobility is rebounding from the pandemic, with services in at least 298 cities and 232,000 vehicles in operation, which is higher than 2019 pre-pandemic numbers. However, while the 128 million trips taken in 2021 is an improvement from 2020, people are still not riding at pre-pandemic levels.

Other findings from the report include:
• Electric-assist e-bikes thrived in 2021, with the number of trips increasing from 9.9 million to 18.8 million.
• Trips on e-scooters accounted for 49% of total trips taken on shared micromobility.
• About 37% of shared micromobility trips replaced a car trip, which has helped to offset 54 million pounds of CO2 emissions.
• 63% of riders said they used micromobility to connect to public transit

Scooting right along (plus ebikes)
If Bladerunner’s Rick Deckard rode an e-bike, it would look like the NIU SQi prototype.

Hummingbird partnered with engineering firm Prodrive to create a plant-based e-bike called the flax Hummingbird.


McKinsey released a report that shows investors have dumped $8.4 billion into shared micromobility companies. Investment in European micromobility businesses quadrupled since 2020 compared to 2018-2019, and that really may be the space to watch, considering Europe has far more dense cities than the U.S. does.

Tier launched its sixth-generation e-scooter, featuring a wider base, nicer turn signals and better suspension.

The U.K.’s Department of Transport reports weekday cycling rates have jumped 47% in the first half of 2022.

Vermont is the latest state to step in where the federal government has not and launched an e-bike incentive program. The state has $100,000 worth of funding to offer residents up to $250 for a standard rebate for income-qualifying purchasers and as much as $400 for an enhanced rebate. The amount will depend on the individual’s adjusted gross income and the price of the e-bike. It’s on a first-come, first-served basis, so act quickly if you live there!


Bike shop owners are struggling to get rid of inventory. Times sure have changed since the heady days of the pandemic when you couldn’t find a bike to buy anywhere.'''

In [ ]:
txt7 = '''Freight transportation-focused Rose Rocket recently raised a $25 million round.

The Toronto-based startup calls the round a Series A, with Addition Capital and Shine Capital leading the deal.

Per venture data aggregated by Carta, average logistics-themed Series A rounds since the start of 2020 have sported median cash raises of $8 million and post-money valuations of around $50 million. The Rose Rocket round certainly surpasses its sector median in dollar terms, and we presume if measured by valuation as well.


But the money side of the Rose Rocket story isn’t the most interesting thing about the company. Its market is.

The startup is building in the TMS space. What’s a TMS? A good question, and one that I had to ask while digging into Rose Rocket. TMS stands for transportation management software, which you can think of as a wide array of products that help truckers and brokers and other players in the freight transport game get work done on a digital basis.

Or, as Rose Rocket co-founder and CEO Justin Sky explained to TechCrunch in an interview, TMS is a somewhat generic moniker that one might compare to ERP (enterprise resource planning). Sky added that he often describes what Rose Rocket is building as a NetSuite for trucking companies.

The startup’s trick is, in Sky’s phrasing, going “horizontal inside of a vertical.” By that, the CEO means that Rose Rocket wants to coordinate more than a single part of the larger freight trucking landscape; instead, the company wants to coordinate all players, allowing for a more integrated, less chaotic market.

For example, Rose Rocket offers the same software to both freight brokers and trucking companies. If a freight broker sends work to a trucking company and both use the startup’s software, Sky said that the work can proceed as if each company was reading off the same sheet music. Instead, of say, one of the parties playing a Chopin nocturne while the other plunked out a free jazz solo.


Like many vertically themed software plays, Rose Rocket is taking on other market-focused software providers, but also customers’ use of Excel and manual processes. Trucking as an industry, you won’t be utterly shocked to learn, is not precisely fully digitized yet. That could provide a large total addressable market (TAM) for Rose Rocket to sell into given how large trucking is as both a business (and employer).

Results thus far seem promising, with the company claiming that its customer logo count went up by around 9x in the last year. Sure, logo count is far from GAAP, but contracted deal volume also rose by 9x over the same period, so, presumably, the company’s business is doing numbers.

That Rose Rocket is a Canadian company — albeit one with U.S. incorporation, essentially a requirement to raise money in North America — is notable. The Canadian startup market has come a long way since Shopify broke into the larger business consciousness. We should probably pay it more heed.

Per CB Insights data, Canadian startups have raised some $9.3 billion thus far in 2021, through the end of Q3. That figure is more than what the country’s upstart tech companies managed in 2019 and 2020 combined. The country is also on pace to hit a record number of discrete venture rounds this year. Rose Rocket is therefore part of a growing trend of larger and more frequent startup investment in the country.


More when someone dumps an inevitable $100 million onto Rose Rocket’s head.

'''

In [ ]:
txt8 = '''Regent, a Boston startup making electric seagliders, has raised $18 million in fresh venture funding from Thiel Capital, JAM Fund, Mesa Air Group and others.Its seagliders are designed to motor out of a harbor on a hydrofoil, take off at a low speed using the water as a runway and then fly over the waves at a top speed of 180 miles per hour.Mesa Air Group CEO Jonathan Ornstein says his company plans to purchase 200 of Regent's fast-flying, 12-passenger seagliders to establish new city-to-city direct routes on the coasts. Thiel Capital and regional aviation providers Mesa Air Group are investing in Regent, a start-up building electric seagliders that can fly low over the water at a top speed of 180 miles per hour.Rather than taking off from a runway at an airport, Regent's 12-passenger seagliders motor out of a harbor on a hydrofoil and and can take off at a low speed due to their fixed-wing design. The company wants to make trips between coastal cities fast, safe and affordable with the smallest possible environmental footprint, said CEO and co-founder Billy Thalheimer. (The start-up's name is an acronym for Regional Electric Ground Effect Naval Transport.)
'''

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
list_true = [txt2, txt3, txt4, txt5, txt6, txt7, txt8]

In [ ]:
([get_probability(i) for i in list_true])

In [ ]:
txtt1 = '''John Carmack, the game developer who co-founded id Software and served as Oculus’s CTO, is working on a new venture — and has already attracted capital from some big names.

Carmack said Friday his new artificial general intelligence startup, called Keen Technologies (perhaps a reference to id’s “Commander Keen“), has raised $20 million in a financing round from former GitHub CEO Nat Friedman and Cue founder Daniel Gross. Stripe co-founder and CEO Patrick Collison, Shopify co-founder and CEO Tobi Lütke, and storied venture fund Sequoia are among those investing in the round, he said.


“This is explicitly a focusing effort for me. I could write a $20M check myself, but knowing that other people’s money is on the line engenders a greater sense of discipline and determination. I had talked about that as a possibility for a while, and I am glad Nat pushed me on it,” he said in a tweet.


AGI, short for “artificial general intelligence,” is a category of AI systems that can theoretically perform any task that a human can. That’s as opposed to AI systems today, which are designed for narrow, specific applications like generating art, driving cars, and playing video games. Some optimists believe AGI could be achieved within the next century, thanks to emerging algorithmic techniques and increasingly powerful computer hardware. Others, including the chief scientist at Carmack’s current employer, Yann LeCun, have cast doubt on the notion that AGI will ever be a reality.

For his part, Carmack seems firmly in the optimist camp. Speaking to MIT research scientist Lex Fridman on a recent podcast, he said that he believes AI will someday reach the point where it behaves akin to a human being or living creature, capable of becoming a “universal remote worker.”

“I do not believe in fast takeoffs … [but] I think that we will go from a point where we start seeing [AI-powered] things that credibly look like animals,” Carmack said. “I think animal intelligence is closer to human intelligence than a lot of people like to think and … There’s just that smooth spectrum of how the brain developed and cortexes and scaling of different things going on there.”

Carmack goes on to note that his interest in AGI was in part spurred by Sam Altman, who he says tried to recruit him to join OpenAI. (Altman is one of the original co-founders of OpenAI, the high-profile AI research lab based in San Francisco.) Altman, who’s previously said he expects AI to one day surpass human intelligence, has been vocal about his concerns about AGI — specifically how it might damage society if built or used incorrectly.


Carmack will still continue to consult with Meta on VR matters, but his work with the social juggernaut, he said, will consume only 20% of his time. Carmack moved to a “consulting CTO” position at Oculus in November 2019 to devote more time to researching AI.


Carmack isn’t the only storied technologist to found an ambitious AI startup recently. For roughly a year, LinkedIn co-founder Reid Hoffman and founding DeepMind member Mustafa Suleyman have been building Inflection AI, a startup that seeks to translate human interactions into a language computers can understand. Former VP of research at OpenAI Dario Amodei is among the co-founders of Anthropic, which for most of 2021 and into this year has studied the behaviors of AI text-generating systems like OpenAI’s GPT-3.'''

In [ ]:
txtt2 = '''Based in Japan, biotech startup Dioseve’s ambitious goal is to grow human oocytes, or eggs, from other tissue. Its aim is to help people struggling with infertility, and it recently raised $3 million led by ANRI, with participation from Coral Capital.

Dioseve’s mission might sound like it comes out of science fiction, but it’s based on a scientific technique called induced pluripotent cells stem (iPS) cells, which was first developed in 2006.

The startup’s scientific advisor, Dr. Nobuhiko Hamazaki, a research specialist at the University of Washington, created Dioseve’s technology, called DIOLs (directly induced oocyte-like cells), which can turn iPS cells into oocytes en masse. DIOLs is currently in trials and has been published in scientific journal Nature.


The new funding will enable Dioseve to hire more people and accelerate its research and development. It aims to establish proof of concept by having mice give birth with DIOLs-produced oocytes, and recently established a new lab in Tokyo and hired an iPS specialist.

As Dr. Hamazaki explains, induced pluripotent stem cells can be used to grow all of the cells in the body. For example, other researchers are finding ways to use iPS to grow organs outside of the body, induce beta cells in the pancreas in an attempt to cure diabetes and generate neural stem cells to cure spinal injuries (iPS cells can be made from tissue like muscle or blood cells).

DIOLs creates oocyte-like cells directly from iPS cells without undergoing primordial germ cells, or the source of sperm and oocytes. It differentiates between them to find oogonia, or the precursor of oocytes and then introduces genes into the iPS cells. This means that people who are dealing with infertility can potentially use DIOLs to have offspring with their own genetic material.

Dr. Hamazaki said that in the case of mice, it usually takes 30 days to get oocytes, and that with human oocytes, it can take up to six months.

Dioseve’s CEO is Kazuma Kishida, who became interested in biotechnology when he was diagnosed with hepatitis C as a teenager. At that time, the available treatment had heavy side effects and a low response rate, so his doctor told him to wait a few years, because a new drug was being developed in the United States. After three years, Kishida got the treatment, curing his hepatitis C. “That drug really changed and contributed to the world,” he told TechCrunch. “I wanted to do something that could change the world like the new drug did.”


Kishida said Dioseve has been giving a lot of thought to the safety and ethics of DIOLs by having conversations with potential patients and science and medical ethics specialists. Right now, issues it is monitoring include the inheritance effect of the technology — can it not only produce healthy babies, but also avoid health issues in subsequent generations?

“We are really serious about ethics. We need to be very careful because this technology can be applied to the process of making a child,” said Dr. Hamazaki, adding, “we need to have a deep conversation with society to get a consensus if this is applicable, and the range we can apply this technology.”

Dioseve isn’t the only biotech startup researching ways to grow human oocytes. Others include Ivy Natal and Conception, both based in San Francisco, which are also developing ways to grow eggs from other cells. Dioseve says its competitive edge is its research progress and practicality.'''

In [ ]:
txtt3 = '''There are many startups working on decentralized identities for the next generation of the internet. Four Tencent veterans want their offering .bit, an identity protocol built on the blockchain, to become the universal identification system in web3, like how emails and phone numbers became ubiquitous in Web 2.0, while giving users control over their own data rather than letting it reside in the platforms they use.

.bit-powered identities come in the form of its namesake domain name. This is how it works: pick a .bit alias, link it up with the addresses of your crypto or NFT wallets that .bit currently supports, and all the data and assets from those wallets will now sit under .bit’s “data container” and be displayed on your .bit page.

Say you need to receive cryptocurrencies from someone. Instead of giving them your 35-character wallet address, you could give them something as simple as claire.bit.


The Ethereum Name Service (ENS) is probably the biggest player in the blockchain naming system space today with .eth increasingly seen in Twitter names. While .bit sees ENS as its rival, it doesn’t bill itself as a domain service but rather as an identity solution provider.

Furthermore, Tim Yeoh, one of .bit’s co-founders, said his company provides a “neutral” and “chain-agnostic” solution whereas .eth interacts with only Ethereum.

“In real life, your houses and financial assets are all tied to your unique ID number; .bit serves as the digital ID for all your assets in the web3 world,” said the founder.

One year after its founding, .bit has registered more than 110,000 accounts with some 38,000 crypto addresses tied to them. The identity protocol now supports Polygon, Tron, Binance Smart Chain and Ethereum; it plans to add compatibility with Bitcoin, Dogecoin, Polkadot, Solana and more down the road. All in all, the protocol has integrated with nearly 100 wallets and dApps.

Its business model is simple: charging an annual subscription fee for the accounts, including the domain names and other identification services. The strategy won investor support as the company has closed a Series A funding round of $13 million led by CMB International, which is owned by the Chinese conglomerate China Merchants Group, HashKey Capital, known for its early investment in Ethereum, QingSong Fund, GSR Ventures, GGV Capital and crypto-focused investment firm SNZ. 


The startup plans to spend the capital on expanding partnerships, developing its user community around the world and hiring — though it wants to keep the team small and nimble.

The company’s next ambition is to promote the use of .bit for decentralized autonomous organizations (DAOs), said Yeoh. Every member of the DAO gets assigned their .bit account and can use that piece of identity to vote on the organization’s decisions.

The startup comprises a small team of 10 people spread across the U.S., China and Singapore, led by four co-founders who were colleagues at Tencent — Tim Yoeh, Specer Shaw, Jeff Jin and Kyle Wright.

Its users already span some 180 countries, but Yeoh wants the company to reach more users in Africa and South America, where a large proportion of the population remains unbanked due to their lack of government documentation.


“A person who doesn’t own an official ID can get a .bit account, skip the government and start using a range of apps,” the founder suggested. “If a person makes a contribution to a DAO, that record can be reflected on their .bit profile. There’s no need for certificates anymore.”

Similar to .eth, .bit has been gradually gaining recognition among crypto adopters on some established internet platforms. On Jike, a social network favored by China’s tech workers, venture capitalists and web3 enthusiasts, people are attaching the .bit suffix to their names even if they haven’t actually registered an account with the platform.

“They are treating .bit as a form of social cachet,” said Yeoh.'''

In [ ]:
txtt4 = '''There’s no end to digitizing the operations of small and medium businesses (SMBs) in the traditional retail sector across sub-Saharan Africa. In Nigeria alone, this industry, worth more than $200 billion, consists of over 40 million businesses of varying sizes, according to reports.

Traditional retail in the country comprises small kiosks and open-air markets selling various products from food and beverages and groceries to personal care products and stationery. B2B digital marketplace upstarts such as TradeDepot, Sabi and Omnibiz have raised millions of dollars to help thousands of these businesses purchase inventory from manufacturers while providing solutions to track cash flow, payments and access capital. Meanwhile, another group of startups provides software and apps to assist these retailers with their bookkeeping and sales tracking processes, among other features.

Pastel, a startup in the second category which has been under the radar for more than a year, is announcing a seed raise of $5.5 million led by pan-African venture capital firm TLcom Capital. Other VC firms such as Global Founders Capital (GFC), Golden Palm Investments, DFS Labs, Ulu Ventures, Plug and Play and Soma Cap also participated in the seed round. The startup raised a $620,000 pre-seed last year from some of its existing investors.


Formerly Sabi Cash, Pastel was founded by Abuzar Royesh, Olamide Oladeji and Izunna Okonkwo, Stanford graduate students who, according to co-founder and chief growth officer (CGO) Okonkwo, in an interview with TechCrunch, shared similar interests in building solutions for micro and SMBs in emerging markets, particularly from their countries of origin: Afghanistan (Royesh) and Nigeria (Oladeji and Okonkwo).

The company’s flagship product, Sabi (not to be confused with a B2B e-commerce marketplace with the same name), is a digital bookkeeping app that allows small businesses to monitor and manage their transactions and customers, get insights into their cashflows, issue receipts, and effectively manage customers who owe them.

Small businesses in Nigeria have stayed offline for years, storing information and essential data offhand, on paper or ledgers. All these inefficiencies, asides from being time-consuming, lead to errors and affects cash flow and finance, which is why nine out of 10 small businesses in the country fizzle out in the first five years. Bookkeeping solutions such as Pastel’s help these businesses streamline processes digitally and save money.


L-R: Izunna Okonkwo (CGO), Olamide Oladeji (CPO) and Abuzar Royesh (CEO). Image Credits: Pastel


The bookkeeping and customer relationship management Pastel launched last year recorded more than 100,000 merchant sign-ups by December 2021, said Okonkwo. The free app currently has more than 45,000 active merchant users. Pastel has recently added more features for merchants to capture more value in this chain. However, unlike other platforms that have bundled several features into one app, Pastel chose a different strategy and made each product standalone: Quick Receipt and Pastel Financing.


“Our thought process was to get traction quickly by solving a merchant’s pain point with a free and easy solution. The next step was to capture value. So we added value capture features to the Sabi app that our customers love. Now we are building a lot more,” Okonkwo said about the company’s decision to build three standalone platforms instead of coupling them all into one app. “The way we’ve thought about it is, as opposed to creating a super app that a lot of other fintechs have or are in pursuit of, we are taking a more platform approach, meaning that any Pastel user can create an account with any of our apps. With the same login they can access all the other solutions that we’re providing.”

The Quick Receipt app provides businesses with simple invoicing and receipts tools and over 60,000 active merchant users. On the other hand, the Swift Money app, which leverages local saving groups called ajo in Nigeria to provide financing for businesses, has been stealthily built for the last three months through Pastel Financing.

Ajo or esusu is a popular financial scheme in Nigeria where a group of people contributes money in varying intervals to a leader who stores the cash on their behalf. They can have different purposes for engaging in this activity, such as saving toward a particular goal or accessing a large pool of credit.

The Swift Money app is designed to work with already established ajo leaders and their groups. Pastel does not aim to replace these leaders, but rather provide them with tools to better track their groups’ well-being, as well as provide financing for the group members. In addition to engaging with the formalities of cash collection and depositing cash in a [Pastel] bank account for savings, the leader downloads the app to set up profiles for members. When any of the members (which in this case are merchants) want to access loans, the leader requests on the app, the member is evaluated based on their saving history and if credit worthy, a business loan is disbursed to them.


The leader is then in charge of recollection of these loans to a pastel account similar to the way they have always done their savings collections.

Pastel only started making revenue recently and does so by charging interest and a small fee on these loans; it also has access to savings to use as a float for loan financing. The one-year-old company plans to raise debt capital by January to complement this process, Okonkwo said. “We’ve not raised debt now because we’ve not scaled out our loan product.”

The U.S.-headquartered and Lagos-based Pastel isn’t the only startup operating in this line of business. In the last 24 months, several startups across West Africa, including Kippa, Bumpa (which recently launched a social commerce integration with Meta), OZÉ and Bamba, have ventured to serve small and medium businesses with bookkeeping tools and credit. While they offer almost identical features such as handling bookkeeping, managing inventory and tracking sales, Okonkwo argues that Pastel’s product-centric approach distinguishes the company from others.

“As a team, we are taking a product-led growth approach where we iterate after doing massive amounts of research into how users are using the solutions and what they are asking for,” said the co-founder. The new capital will assist Pastel in increasing its efforts in this area as it looks to expand its product offerings and develop more productivity and finance management features and tools around group savings, loans and payments for small businesses.'''

In [ ]:
txtt5 = '''Startups that are catering to homebuyers are struggling as interest rates and inflation have climbed and inventory shortages continue in many markets. 
The latest casualty in the space is Reali, which announced that it has begun a shutdown and will be laying off most of its workforce on September 9.
In a press release, co-founder and chairman Amit Haller said “the challenging real estate and financial market conditions and unfavorable capital-raising environment” led to the decision to wind down operations.
“Reali was one of the pioneering companies to offer the ‘buy before you sell’ and ‘cash offer’ programs to homeowners,” he said in the release. “We believed deeply in benefiting the consumer foremost in every transaction.”
Amit Haller and Ami Avrahami founded Reali in Israel in 2016 out of personal frustration as real estate investors with the quality of service provided by agents and the high commissions they paid out.
Their goal was to make “the real estate buying and selling process more transparent, honest and professional.”
The startup touted that customers could buy and sell in one coordinated transaction, “eliminating resale contingencies, moving twice and paying two mortgages at once.”
Since inception, the company has raised more than $290 million in debt and equity funding, according to Crunchbase. Its last raise was a $100 million Series B in August of 2021 led by Zeev Ventures with participation from Akkadian Ventures, Signia Ventures and others. At the time of that raise, the company was reported to have 180 employees, according to CTECH by Calcalist. The same publication reported this week that Reali had 140 employees.
In its press release, Reali said that “a small team of employees” would continue to support active real estate transactions through the end of the year.
The company added that it is in discussions with companies that have expressed interest in buying certain parts of its business, including mortgage origination, title & escrow, and power buying.
Haller and Avrahami also founded another startup, Veev, a real estate developer turned tech-enabled homebuilder that in March raised $400 million in a Series D round that propelled the company to “unicorn status.”
Reali is not alone in its challenges. Homeward recently laid off 20% of its workforce. And Better.com, which has had a host of other issues, has laid off thousands this year.
My weekly fintech newsletter, The Interchange, launched on May 1! Sign up here to get it in your inbox.

'''

In [ ]:
txtt6 = '''More companies are adding payments and other financial features to their offerings; however, this often requires technical expertise that some don’t have.

Enter Deposits, a Dallas-based finance startup offering a cloud-based, plug-and-play feature to simplify the implementation of digital banking tools for companies like credit unions, community banks, insurers, retailers and brands.

Co-founder and CEO Joseph Akintolayo started the company with Daniel Paramo in 2019 after years of working with banks and listening to people describe how they want their money to work.


“They are often describing something that sounds kind of like a credit union,” he told TechCrunch. “They want to feel ownership. They want to make sure that it supports their community. They want to feel in charge and that it’s personalized and communicates with them.”

Deposits digital banking tools credit unions
Deposits’ app features Image Credits: Deposits


When he looked at what his clients on the agency side could provide, he saw a big gap between what people were asking for and what banks were able to provide. For example, only the biggest of banks could really offer modern experiences. However, that comes at a big cost to people who are underbanked or who can’t afford monthly fees, Akintolayo explained.

“What’s left are people with a ‘will,’ and community banks and credit unions without a ‘way,’” he added. “Deposits was born to bridge community institutions and brands that really have a passion for serving consumers with the tools that they actually needed to serve consumers in a modern fashion.”

The company enables its customers to quickly and easily put together a package of features, including mobile apps, debit and credit accounts, mobile deposits, virtual cards, peer-to-peer payments and online accounts with identity verification. They can also offer services like home and auto loans and foreign exchange.


Deposits partners with a number of banking, credit and payments partners to facilitate its series of modular “kits” using no-code or low-code tools and APIs that include business banking, money management, identity verification, embedded finance for shopping and checkout and workplace needs.



The company joins other fintech companies catering to credit unions and small banks, like Narmi, Bankjoy and Xend Finance in receiving venture capital funding for their approaches. In Deposits’ case, it announced today its first investment, a $5 million seed round led by ATX Venture Partners with participation from Cabal Fund, Lightspeed Venture Partners and others.

“The first wave of embedded fintech platforms demonstrated demand but required deep technical expertise and suffered delayed rollouts,” said Chris Shonk, partner and co-founder at ATX Venture Partners, in a written statement. “Deposits delivers on the promise with a low-code platform that literally any credit union or product brand can use. Combine that with the team Joseph has put together, its compelling mission and impressive early traction, it was an easy decision to invest.”

The company’s new capital infusion will be deployed into product development and adding to its team of 27 across sales, marketing and engineering, Akintolayo said. He recently brought on a vice president of business development and will be rounding out the executive team.


Deposits’ platform is still in the early stages, having just launched a little over a year ago; however, Akintolayo said it is already “showing strong signals” that it is ready for a broader market.

“Business has not slowed down for us,” he added. “We’re very bullish on our segment and supporting a very needed and essential part of the economy. The products and services that we offer are needed in both time of plenty and in time of need, and so we feel very good.”'''

In [ ]:
txtt7 = '''
 Sports brand Adidas opened a speed-factory in Germany, which is the ﬁrst automated factory that quickly produces a small batch of designs for speciﬁc markets, which cuts the production time from months into a single day. The reason behind this great technology and innovation, is that consumers shop from everywhere. “In today’s world, where you’re more likely to order a pair of Yeezys from the back of an Uber, than in a store, the gap between wanting something and expecting to have it has shrunk down to almost nothing.” – Rowland Manthrope, associate editor of Wired UK Technology advances and is making it possible for humans with disabilities to function completely independently in todays society… Be My Eyes and Aira – two companies which pair the blind to an assistant to trained visual interpreters. Assistance includes navigation, reading, instructions, and directions. Vision is becoming a strong target-market, Microsoft’s Garage’s hearing AI uses easily understandable graphics to illustrate sound, and shows their intensity and the volume.
 Brands are ﬁnally recognising roles that can be distributed in this market, after following Toyota which launched a wearable device for the blind in 2016. Chinese tech giants go global…
 After growing massively behind China’s Firewall, they are now able to venture out to the rest of the world. Alibaba and Tencent offer access to almost 400 million Chinese households. Alibaba founder, Jack Ma, says his company could help one million American businesses to sell to China and the rest of Asia within the next ﬁve years. Yet western countries remain sceptical when it comes to China promising data privacy and security while being under pressure from their government. This expansion of Asian Tech is coming in a time period when America is slowly and quietly slowing down in the race. Quite worth following, because if Asia does manage to rebrand to focus on the western world, they will make a huge impact on the tech landscape. Watching the huge tech backlash after the 2017 president election of Donald Trump, when Facebook and Twitter were used to inﬂuence voters… A huge storm of bad PR for the tech sector hit especially the US which was full of allegations of fake news, gender discrimination and many others. While Mark Zuckerberg’s hearing in congress happened a few weeks back, for the whole world to watch, executives from Facebook, Google and Twitter appeared before Congress in relation to alleged Russian use of their platforms to manipulate the US election. What will happen is – big tech ﬁrms will be closely monitored by governments and regulators in the years to come, while many experts also think that “tech giants might actually shift or be disrupted within a decade or two.” – StJohn Deakins, founder and CEO of a personal data exchange platform, CitizenMe. Our Airbnb-Everything-society… Airbnb is the largest disrupter of the hotel industry. There are hundreds of Apps that follow the same ways of living, working, travelling, commuting, dining etc. – We are developing into an almost complete sharing-economy. It’s attractive, it seems lower budget, people are not obliged to pay membership fees, rent, car insurances, or ofﬁce spaces anymore. It can all be done, while being the place you have always craved to live, and by working remotely. But back to Airbnb, which now has bigger plans than ever. Back in 2016 they already launched Trips, which gave the hosts of the platform to offer local experiences as well. It also includes experiences and local guides which future sections planned for ﬂights and services. In September 2017, Airbnb partnered with the App Resy to offer in-app restaurant reservations and reviews. In the same time period, they also announced they are planning on launching their own branded apartments and now, in 2018 the ﬁrst 300-unit building is due to open, in Florida, US. The building is aimed to the market of frequent travellers and is secured with storages, and on-site-cleaner to make the subletting through Airbnb way more simple.
'''

In [ ]:
txtt8 = '''Have you ever thought about applying with your startup for funding from the European Union? Well, there are some great opportunities, butregarding bureaucratic hurdles and the proposal writing we’d suggest to work with some experts. EU Startup Services helps startups and SMEs to secure European Union funding through the Horizon 2020 SME instrument. One of their most recent successes was that they secured a grant of €2.48 million– close tothe maximum amountallowed under the SME Instrument Phase 2 -,forMobius Protection Systems, an Israeli company, for their disruptive innovation –SAFESEAT– toreduce child casualties and injuries in traffic collisions. The Mobius CEO stated: “We chose EU Startup Services for their expertise and impressive track record in the SME Instrument. We are delighted to say that we made the right choice. I highly recommend working with EU Startup Services, not only for their ability to write, but also for theirinsider knowledge and their ability to foresee and remedy structural challenges faced in our project design which ultimately made our project unbeatable.” Mobius’ proposal received a score of 14.10 out of 15 under the topic “Small business innovation research for Transport and Smart Cities Mobility” at the June 2017 cut-off. Does your startup also work on a disruptive and/or innovation idea/product? EU Startup Services helped more than 31 startups so far to secure EU Funding.Your startup might be the next one!
'''

In [ ]:
list_false = [txtt1, txtt2, txtt3, txtt4, txtt5, txtt6, txtt7, txtt8]

In [ ]:
[get_probability(i) for i in list_false]

In [ ]:
import torch
torch.save(model, './content_model')
saved_model = torch.load('./content_model')

In [ ]:
def get_probability_x(text):
  new_sentence = text

  # We need Token IDs and Attention Mask for inference on the new sentence
  test_ids = []
  test_attention_mask = []
  # Apply the tokenizer
  encoding = preprocessing(new_sentence, tokenizer)
  # Extract IDs and Attention Mask
  test_ids.append(encoding['input_ids'])
  test_attention_mask.append(encoding['attention_mask'])
  test_ids = torch.cat(test_ids, dim = 0)
  test_attention_mask = torch.cat(test_attention_mask, dim = 0)

  # Forward pass, calculate logit predictions
  with torch.no_grad():
    output = saved_model(test_ids.to(device), token_type_ids = None, attention_mask = test_attention_mask.to(device))
  prediction = 'transport' if np.argmax(output.logits.cpu().numpy()).flatten().item() == 1 else 'no transport'
  # print("---")
  # print(output)
  # print(torch.nn.functional.softmax(output.logits.cpu())) # ham / spam
  # print("---")
  # print('Input Sentence: ', new_sentence)
  # print('Predicted Class: ', prediction)
  probs = torch.nn.functional.softmax(output.logits.cpu()) 
  return prediction

In [ ]:
[get_probability_x(i) for i in list_false]

In [ ]:
[get_probability_x(i) for i in list_true]